##### Project: Opioid Exposed Infant Covariates
##### Investigator: Stephen Patrick, Sarah Loch
##### Programmers: Sander Su, Chris Guardo
##### Date Created: 01/17/23
##### Last Modified: 09/30/25

#### Notes: 
This step comes after birth visit in the workspace pipeline. It creates a set of codes matching drug related codes in the infants’ electronic health record with the birth hospital visit. 



In [0]:
%run "../Project_modules"

#### Search for infant drug exposure records:
##### infant drug list:
##### - morphine
##### - methadone
##### - phenobarbital
##### - clonidine
##### cohort: 
##### - infant in phenotype cohort
##### - time limit: birth hospitalization


In [0]:
phenotype_table_location = " ***Insert file location*** "
phenotype_table=spark.sql(f"SELECT * FROM {phenotype_table_location}")

In [0]:
sql=f"""
       select * from {phenotype_table} a
       inner join 
       (select * from {drug_exp_table}
       where lower(trim(drug_source_value)) like '%morphine%' 
       or lower(trim(drug_source_value)) like '%methadone%' 
       or lower(trim(drug_source_value)) like '%phenobarbital%' 
       or lower(trim(drug_source_value)) like '%clonidine%') b
       on a.baby_person_id = b.person_id;
    """
mombabypair_infant_drug = spark.sql(sql)
mombabypair_infant_drug.createOrReplaceTempView("mombabypair_infant_drug")

##### Validation

In [0]:
#df_inspection("mombabypair_infant_drug","baby")

##### time limit: birth hospitalization

In [0]:
sql="""
   select distinct infant_drug.*,a.first_visit_start_date, a.first_visit_end_date from 
   (select * from global_temp.mom_baby_step1_baby1stvisit) a
   inner join mombabypair_infant_drug infant_drug
   on a.baby_person_id = infant_drug.person_id
   where drug_exposure_start_date >= first_visit_start_date and drug_exposure_start_date<= first_visit_end_date;
   """
mom_baby_step1_baby1stvisit_infant_drug= spark.sql(sql)
mom_baby_step1_baby1stvisit_infant_drug.createOrReplaceTempView("mom_baby_step1_baby1stvisit_infant_drug")

##### Adding the information in table 'x_drug_exposure', especially the dose info

In [0]:
sql=f"""
      select a.*,b.person_id as dose_person_id, b.drug_exposure_id as dose_drug_exposure_id,b.x_doc_type,b.x_doc_stype,
      b.x_dose,b.x_drug_form,b.x_strength,b.x_frequency
    
      from mom_baby_step1_baby1stvisit_infant_drug a left join {drug_exp_table_extra} b 
      on a.drug_exposure_id = b.drug_exposure_id;
    """
mom_baby_step1_baby1stvisit_infantdrug_dose = spark.sql(sql)
mom_baby_step1_baby1stvisit_infantdrug_dose.name='mom_baby_step1_baby1stvisit_infantdrug_dose'
register_parquet_global_view(mom_baby_step1_baby1stvisit_infantdrug_dose)

In [0]:
# Display df so can export
mom_baby_step1_baby1stvisit_infantdrug_dose.display()

#### Validation

In [0]:
df_inspection("global_temp.mom_baby_step1_baby1stvisit_infantdrug_dose","baby")



### Save Output for future use

In [0]:
mom_baby_step1_baby1stvisit_infantdrug_dose.write.mode("overwrite").saveAsTable(f"covariate_output.mom_baby_1st_visit_infantdrug_dose")